In [ ]:
!pip install -q pyomo
from pyomo.environ import * 
import numpy as np
import pandas as pd
!apt-get install -y -qq coinor-cbc

In [3]:
coef = np.loadtxt('lab4_ex2_lp_ip_coef.txt', delimiter=',')

model=ConcreteModel()

N=coef.shape[1]-1
M=coef.shape[0]-1

obj_coef = coef[0,:-1]
constr_coef = coef[1:,:-1]

constr_rhs = coef[1:,-1]

col_indices = np.arange(N)

model.x = Var(col_indices,domain=Binary)

row_indices = np.arange(M)

model.constraints = ConstraintList()

for i in row_indices:
  model.constraints.add(summation(constr_coef[i],model.x) <= constr_rhs[i])

model.objective = Objective(expr = summation(obj_coef,model.x), sense=maximize)

opt_cbc = SolverFactory('cbc')

result = opt_cbc.solve(model)

print('\nObjective = ', model.objective())
print("\nsolver time",result.solver.time)
print("solver statistics",result.solver.statistics)


Objective =  295828.0

solver time 65.8500497341156
solver statistics 
Branch and bound: 
  Number of bounded subproblems: 347044
  Number of created subproblems: 347044
Black box: 
  Number of iterations: 1407089



In [6]:
#solving corresponding LP problem by removing the integer restrictions from decision variables

model.x.domain=NonNegativeReals

lower_bound = 0
upper_bound = 1

model.x.setlb(lower_bound)
model.x.setub(upper_bound)

opt_cbc = SolverFactory('cbc')

result = opt_cbc.solve(model)

print('\nObjective = ', model.objective())
print("\nsolver time",result.solver.time)
print("solver statistics",result.solver.statistics)



Objective =  295896.37697236

solver time 0.024948835372924805
solver statistics 
Branch and bound: 
  Number of bounded subproblems: None
  Number of created subproblems: None
Black box: 
  Number of iterations: 18



Comparing the time in two cases   
MILP solver time is 65.85 while LP solver time is approximately 0.025. So time taken in solving MILP is approximately 2600 times more than that taken in solving LP.